In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bankcustomersdata/BankCustomers.csv


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [3]:
df = pd.read_csv("/kaggle/input/bankcustomersdata/BankCustomers.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [5]:
df = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


In [6]:
genders_df = pd.get_dummies(df['Gender'], drop_first=True)
geography_df = pd.get_dummies(df['Geography'], drop_first=True)
df = pd.concat([df, genders_df,geography_df], axis=1)
df = df.drop(['Gender', 'Geography'], axis=1)
df.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Male,Germany,Spain
0,619,42,2,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,0,1
2,502,42,8,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,0,1


In [7]:
df_x = df.drop(['Exited'], axis=1)
df_y = df['Exited']

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_x.values, df_y.values, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(8000, 11) (2000, 11) (8000,) (2000,)


In [9]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
print(X_test[:5])

[[-0.57749609 -0.6557859  -0.69539349  0.32993735  0.80843615 -1.54035103
  -1.02583358 -1.01960511  0.91324755  1.72572313 -0.57638802]
 [-0.29729735  0.3900109  -1.38944225 -1.21847056  0.80843615  0.64920267
   0.97481699  0.79888291  0.91324755 -0.57946723 -0.57638802]
 [-0.52560743  0.48508334 -0.3483691  -1.21847056  0.80843615  0.64920267
  -1.02583358 -0.72797953 -1.09499335 -0.57946723  1.73494238]
 [-1.51149188  1.91116988  1.03972843  0.68927246  0.80843615  0.64920267
   0.97481699  1.22138664  0.91324755  1.72572313 -0.57638802]
 [-0.9510944  -1.13114808  0.69270405  0.78283876 -0.91668767  0.64920267
   0.97481699  0.24756011 -1.09499335 -0.57946723  1.73494238]]


In [10]:
tf.keras.layers.Dense(8, activation='relu')

model = tf.keras.Sequential([
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(5, activation='relu'),
    tf.keras.layers.Dense(3, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


In [11]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [12]:
batch_size = 32
no_of_epochs = 50
model.fit(X_train, y_train, batch_size=batch_size, epochs=no_of_epochs)

Epoch 1/50
250/250 [==============================] - 1s 1ms/step - loss: 0.7054 - accuracy: 0.6404
Epoch 2/50
250/250 [==============================] - 0s 1ms/step - loss: 0.4993 - accuracy: 0.8033
Epoch 3/50
250/250 [==============================] - 0s 1ms/step - loss: 0.4441 - accuracy: 0.8041
Epoch 4/50
250/250 [==============================] - 0s 1ms/step - loss: 0.4261 - accuracy: 0.8162
Epoch 5/50
250/250 [==============================] - 0s 1ms/step - loss: 0.4210 - accuracy: 0.8195
Epoch 6/50
250/250 [==============================] - 0s 1ms/step - loss: 0.4044 - accuracy: 0.8357
Epoch 7/50
250/250 [==============================] - 0s 1ms/step - loss: 0.3815 - accuracy: 0.8385
Epoch 8/50
250/250 [==============================] - 0s 1ms/step - loss: 0.3647 - accuracy: 0.8433
Epoch 9/50
250/250 [==============================] - 0s 1ms/step - loss: 0.3606 - accuracy: 0.8469
Epoch 10/50
250/250 [==============================] - 0s 1ms/step - loss: 0.3505 - accuracy: 0.8517

In [13]:
preds = model.predict(X_test)
preds_values = preds > 0.5
print(preds_values[:5])

[[False]
 [False]
 [False]
 [False]
 [False]]


In [14]:
from sklearn.metrics import confusion_matrix, accuracy_score
print("Confusion Matrix: ",confusion_matrix(y_test, preds_values))
print("Accuracy: ",accuracy_score(y_test, preds_values))

Confusion Matrix:  [[1544   63]
 [ 217  176]]
Accuracy:  0.86
